## Comparison of execution of VerticalPvConfiguration and DefaultConfiguration

In this notebook, the values from the literature for phi, alpha, theta, and p_max are used.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

from agrivoltaics_supply_side_management.agriculture.crops import Cultivation
from agrivoltaics_supply_side_management.configuration \
    import BifacialVerticalPvConfiguration, DefaultBifacialConfiguration
from agrivoltaics_supply_side_management.optimization.convex_optimization\
    import ConvexOptimization
from agrivoltaics_supply_side_management.photovoltaics.pv_modules\
    import BifacialElectricityGeneration
from agrivoltaics_supply_side_management.solar_irradiation.irradiance\
    import BifacialIrradianceManager

In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
phi, alpha, theta, p_max = 0.055, 0.2, 0.96, 15

In [5]:
lattitude, longitude = 49.26757152616243, -123.25266177347093
timezone = 'Canada/Pacific'
time_range = pd.date_range('2022-07-06', '2022-07-07', freq='1T',
                             tz=timezone)

In [6]:
surface_azimuth = 180
surface_tilt = 20
axis_azimuth = 180
pvrow_height = 1
pvrow_width = 4
pitch = 10
gcr = pvrow_width / pitch
albedo = 0.2
n_pvrows = 3
index_observed_pvrow = 1
bifaciality = 0.75

In [7]:
bifacial_irradiance_manager = BifacialIrradianceManager(
    lattitude, longitude, timezone, time_range,
    surface_azimuth, surface_tilt, axis_azimuth, gcr,
    pvrow_height, pvrow_width, albedo, n_pvrows, index_observed_pvrow,
    bifaciality)

In [8]:
optimization = ConvexOptimization()

In [9]:
temp_model_parameters_type, module_name, inverter_name = (
    'open_rack_glass_glass', 'Trina_Solar_TSM_300DEG5C_07_II_',
    'ABB__MICRO_0_25_I_OUTD_US_208__208V_')

In [10]:
bifacial_electricity_generation = BifacialElectricityGeneration(
    lattitude, longitude, timezone,
    bifacial_irradiance_manager.bifacial_irradiances,
    temp_model_parameters_type,
    module_name, inverter_name,
    surface_tilt, surface_azimuth, 
    bifaciality)

In [11]:
harvest_index = 0.95
biomass_energy_ratio = 30
leaf_area_index = 5
crop_growth_regulating_factor = 0.95
cultivation = Cultivation(harvest_index, biomass_energy_ratio,
                          leaf_area_index, crop_growth_regulating_factor)

In [12]:
net_photosynthetic_rate_parameters = {
                "phi": phi,
                "alpha": alpha,
                "theta": theta,
                "p_max": p_max
            }

In [13]:
ppfd_data = np.linspace(0, 1200)

In [14]:
bifacial_vertical_pv_configuration = BifacialVerticalPvConfiguration(
    net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, optimization, 
    bifacial_electricity_generation,
    cultivation)

In [15]:
vertical_electricity_supply, vertical_crop_yield,\
    vertical_cumulative_electric_power_for_morning_peak, \
    vertical_cumulative_electric_power_for_afternoon_peak \
        = bifacial_vertical_pv_configuration.supply(time_range)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpk0_nusf4.glpk.raw
 --wglp /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpruft1ryb.glpk.glp
 --cpxlp /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpiufpky8u.pyomo.lp
Reading problem data from '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpiufpky8u.pyomo.lp'...
3 rows, 3 columns, 4 non-zeros
25 lines were read
Writing problem data to '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpruft1ryb.glpk.glp'...
18 lines were written
GLPK Simplex Optimizer 5.0
3 rows, 3 columns, 4 non-zeros
Preprocessing...
~     0: obj =   0.000000000e+00  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (32525 bytes)
Writing basic solution to '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmpk0_nusf4.glpk.raw'...
15 lines were written
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the comman

In [16]:
vertical_electricity_supply

1625.4660470528108

In [17]:
vertical_crop_yield

0.002696079158555422

In [18]:
vertical_cumulative_electric_power_for_morning_peak

110.52856070040848

In [19]:
vertical_cumulative_electric_power_for_afternoon_peak

520.5982733272864

In [20]:
default_bifacial_configuration = DefaultBifacialConfiguration(
    net_photosynthetic_rate_parameters, ppfd_data,
    bifacial_irradiance_manager, optimization, 
    bifacial_electricity_generation,
    cultivation)

In [21]:
default_electricity_supply, default_crop_yield, \
    default_cumulative_electric_power_for_morning_peak, \
    default_cumulative_electric_power_for_afternoon_peak \
        = default_bifacial_configuration.supply(time_range)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmp18a6v624.glpk.raw
 --wglp /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmp29vfc1gc.glpk.glp
 --cpxlp /var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmps_gjl5od.pyomo.lp
Reading problem data from '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmps_gjl5od.pyomo.lp'...
3 rows, 3 columns, 4 non-zeros
25 lines were read
Writing problem data to '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmp29vfc1gc.glpk.glp'...
18 lines were written
GLPK Simplex Optimizer 5.0
3 rows, 3 columns, 4 non-zeros
Preprocessing...
~     0: obj =   0.000000000e+00  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (32525 bytes)
Writing basic solution to '/var/folders/8n/f15shvd92vv8t4gykc82yrdm0000gn/T/tmp18a6v624.glpk.raw'...
15 lines were written
GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the comman

In [21]:
default_electricity_supply

1559.0330007243103

In [22]:
default_crop_yield

0.0035565037520898677

In [23]:
default_cumulative_electric_power_for_morning_peak

232.54318022331466

In [24]:
default_cumulative_electric_power_for_afternoon_peak

332.1506074758781

### Comparison

In [40]:
vertical_electricity_supply / default_electricity_supply

1.0856838776122162

In [41]:
vertical_crop_yield / default_crop_yield

0.9750073468541308

In [42]:
vertical_cumulative_electric_power_for_morning_peak / default_cumulative_electric_power_for_morning_peak

2.2601221445140554

In [43]:
vertical_cumulative_electric_power_for_afternoon_peak / default_cumulative_electric_power_for_afternoon_peak

1.7387489696676843